In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv(r'complaints_fake.csv')

In [6]:
df.columns

Index(['client_name', 'client_region', 'theme', 'complaint_date',
       'complaint_text'],
      dtype='object')

In [3]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from uuid import uuid4

from langchain_core.documents import Document

In [5]:
documents = []

In [11]:
for index, row in df.iterrows():
    document_temp = Document(page_content=row['complaint_text'],metadata={"client_name": row['client_name'],"client_region": row['client_region'],"theme": row['theme'],"complaint_date": row['complaint_date']})
    documents.append(document_temp)


In [14]:
from langchain.embeddings import BedrockEmbeddings
import boto3
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name='us-east-1',
)

In [15]:
embeddings = BedrockEmbeddings(
            client=bedrock_client, 
            model_id="amazon.titan-embed-text-v2:0"
        )


/tmp/ipykernel_6094/2076088525.py:1: LangChainDeprecationWarning: The class `BedrockEmbeddings` was deprecated in LangChain 0.2.11 and will be removed in 1.0. An updated version of the class exists in the langchain-aws package and should be used instead. To use it run `pip install -U langchain-aws` and import as `from langchain_aws import BedrockEmbeddings`.
  embeddings = BedrockEmbeddings(


In [16]:
vector_store = FAISS.from_documents(documents=documents,embedding=embeddings)

In [17]:
vector_store.save_local('complaints.vs')